<img src="../core/pn_i32.png">
<p style="text-align: center; font-size: 180%; font-weight: bold; margin-top: 10px;">CVI5 Python - Utilitários - Leitura e Importação do Safic</p>

## ✅ Imports

In [44]:
import set_sys_path # este sempre antes de todo mundo, é para acertar certinho o sys.path
# TODAS AS BIBLIOTECAS QUE VOU UTILIZAR
import ipywidgets as widgets
import sqlite3
import os
import random
import datetime
import win32com.client
import core.config as config
from core.pycvi import cvi
from core.pycpfcnpj import cnpj

In [56]:
def adoDb_info():
    today = datetime.date.today()
    uniqid = today.strftime("%y%m%d") + str(random.randint(10_000_000, 99_999_999))
    tmp_file = os.path.join(config.settings.CVI_VAR, 'tmp', f"sqldbi{uniqid}.txt")
    cmd = f"sqllocaldb i SaficV150 > {tmp_file}"
    o_exec = os.system(f"cmd /C {cmd}")
    # print("uniqid=", uniqid, tmp_file, cmd, "#", o_exec, "#")
    with open(tmp_file) as f:
        lines = f.readlines()
    adoDb_info_pipeName = lines[14].replace('Instance pipe name:', '').strip()
    adoDb_info_state = lines[10].replace('State:              ', '').strip()
    return adoDb_info_pipeName, adoDb_info_state


def adoDb_open():
    adoDb_status = adoDb_info()
    if adoDb_status[1] != "Running":
        print("Falha!! SqlLocalDb SaficV150 parado... Movimente o Safic e tente novamente")
        return False
    print(f"SqlLocalDb em funcionamento! State: {adoDb_status[1]} - Pipe Name: {adoDb_status[0]}")
    serverName = '{' + adoDb_status[0] + '};Database={master};Trusted_Connection={True};';
    dsn = 'DRIVER={SQL Server};SERVER=' + serverName + ''
    cn = win32com.client.Dispatch("ADODB.Connection")
    rs = win32com.client.Dispatch("ADODB.Recordset")
    cn.Open(dsn)
    return cn, rs


def adoDb_get_databases():
    cn, rs = adoDb_open()
    databases = []
    sql = "SELECT QUOTENAME(name) AS f1 FROM sys.databases ORDER BY name"
    # Ver instruções completas aqui: https://wiki.python.org.br/PythonComADO
    rs.Open(sql, cn, 2, 3)
    while not rs.Eof:
        databases.append(rs.Fields['f1'].value)
        rs.MoveNext()
    return databases


def adoDb_get_osflist():
    cn, rs = adoDb_open()
    osflist = []
    sql = "SELECT numOsf, identificadorAuditoria, razao, cnpj, ie FROM [SaficV1_0].[dbo].[Auditoria]"
    # Ver instruções completas aqui: https://wiki.python.org.br/PythonComADO
    rs.Open(sql, cn, 2, 3)
    while not rs.Eof:
        osf = [rs.Fields['numOsf'].value, rs.Fields['identificadorAuditoria'].value, rs.Fields['razao'].value, rs.Fields['cnpj'].value, rs.Fields['ie'].value, ]
        osflist.append(osf)
        rs.MoveNext()
    return osflist


def adoDb_get_table_list():
    cn, rs = adoDb_open()
    table_list = []
    val = '[SaficV1_0]'
    sql = f"""
    SELECT '{val}' AS DB, sSCH.name AS SchemaName, sOBJ.name AS TableName, sum(sPTN.Rows) AS QtdLinhas
    FROM {val}.sys.objects AS sOBJ
    INNER JOIN {val}.sys.partitions AS sPTN ON sOBJ.object_id = sPTN.object_id
    INNER JOIN {val}.sys.schemas AS sSCH ON sOBJ.schema_id = sSCH.schema_id
    WHERE sOBJ.type = 'U' AND sOBJ.is_ms_shipped = 0x0 AND index_id < 2 
    GROUP BY sSCH.name, sOBJ.name
    """
    print(sql)
    rs.Open(sql, cn, 2, 3)
    while not rs.Eof:
        res = [rs.Fields['DB'].value, rs.Fields['SchemaName'].value, rs.Fields['TableName'].value, rs.Fields['QtdLinhas'].value]
        table_list.append(res)
        rs.MoveNext()
    return table_list



print(adoDb_get_databases())
print(adoDb_get_osflist())
print(adoDb_get_table_list())

SqlLocalDb em funcionamento! State: Running - Pipe Name: np:\\.\pipe\LOCALDB#A3427B12\tsql\query
['[13003915228_221226_183400]', '[13003915228_221226_183400_Dfe_000101_171031]', '[13003915228_221226_183400_Dfe_171101_210201]', '[13003915228_221226_183400_Dfe_210202_221226]', '[13003915228_221226_183400_DocAtrib_000101_171031]', '[13003915228_221226_183400_DocAtrib_171101_210201]', '[13003915228_221226_183400_DocAtrib_210202_221226]', '[13003915228_221226_183400Aux1]', '[13004146228_230102_115041]', '[13004146228_230102_115041_Dfe_000101_160731]', '[13004146228_230102_115041_Dfe_160801_180201]', '[13004146228_230102_115041_Dfe_180202_230102]', '[13004146228_230102_115041_DocAtrib_000101_160731]', '[13004146228_230102_115041_DocAtrib_160801_180201]', '[13004146228_230102_115041_DocAtrib_180202_230102]', '[13004146228_230102_115041Aux1]', '[master]', '[model]', '[msdb]', '[SaficV1_0]', '[tempdb]']
SqlLocalDb em funcionamento! State: Running - Pipe Name: np:\\.\pipe\LOCALDB#A3427B12\tsql\q